# 0. Load Data

SRX351237_calculus_sra_data.fastq – dental calculus (sample SRX351237) as our experimental sample
SRX351242_bone_sra_data.fastq – tooth root (sample SRX351242) as a proxy for an environmental control

In [ ]:
!wget -O data/SRX351237_calculus_sra_data.fastq https://ndownloader.figshare.com/files/22344444
!wget -O data/SRX351242_bone_sra_data.fastq https://ndownloader.figshare.com/files/22344441

# Part 1. Amplicon sequencing

## Inporting data

We've downloaded manifest file for Qiime2 and change path to the data in it

Import data to Qiime2

`qiime tools import --type 'SampleData[SequencesWithQuality]' --input-path manifest.tsv --output-path data/sequences.qza --input-format SingleEndFastqManifestPhred33V2`


Check correctness of QIIME artifact with qiime validate:

`qiime tools validate data/sequences.qza`

Output:

`Result data/sequences.qza appears to be valid at level=max.`

## Summary of the distribution of sequence qualities

`qiime demux summarize   --i-data data/sequences.qza   --o-visualization data/sequences.qzv`

Visualise it with: https://view.qiime2.org/

Result:

<img src="img/1.png">

<img src="img/2.png">

## Feature table construction

Download metadata table:

`wget -O data/sample-metadata.tsv https://ndownloader.figshare.com/files/22346028`

Strip out barcode and filter chimeric sequences. We use DADA2 pipeline. 

We choose value for parameter `--p-trim-left` as `32` based on length of the artificial sequences (barcode +primer)

and value for parameter `--p-trunc-len` as `148` based on Interactive Quality Plot tab in the sequences.qzv

`qiime dada2 denoise-single --i-demultiplexed-seqs data/sequences.qza --p-trim-left 32 --p-trunc-len 148 --o-representative-sequences data/rep-seqs.qza --o-table data/table.qza --o-denoising-stats data/stats.qza`

Check how many reads are passed the filter and were clustered:

`qiime metadata tabulate --m-input-file data/stats.qza --o-visualization data/stats.qzv`

Result:

<img src="img/4.png">

## FeatureTable and FeatureData summaries

Create visual summaries of the data - how many sequences are associated with each sample and with each feature, etc.

`qiime feature-table summarize --i-table data/table.qza --o-visualization data/table.qzv --m-sample-metadata-file data/sample-metadata.tsv`

Map feature IDs to sequences

`qiime feature-table tabulate-seqs --i-data data/rep-seqs.qza --o-visualization data/rep-seqs.qzv`

## Taxonomic analysis

Download database:

`wget -P data/ https://data.qiime2.org/2021.2/common/gg-13-8-99-nb-classifier.qza`

Use classifier:

`qiime feature-classifier classify-sklearn --i-classifier data/gg-13-8-99-nb-classifier.qza --i-reads data/rep-seqs.qza --o-classification data/taxonomy.qza`

Visualise:

`qiime metadata tabulate --m-input-file data/taxonomy.qza --o-visualization data/taxonomy.qzv`

View the taxonomic composition of our samples with interactive bar plots

```
qiime taxa barplot \
  --i-table data/table.qza \
  --i-taxonomy data/taxonomy.qza \
  --m-metadata-file data/sample-metadata.tsv \
  --o-visualization data/taxa-bar-plots.qzv
```

`qiime tools view data/taxa-bar-plots.qzv`

Result:

<img src="img/level-7-bars.svg">

# Part 2. Shotgun sequencing

## Shotgun sequence data profiling

align our sequencing reads to the microbiota database:

`MetaPhlAn2/metaphlan2.py data/G12_assembly.fna.gz --input_type fasta --nproc 4 > data/metaphlan_output.txt`

Visualuse with Pavian:

<img src="img/5.png">

## Comparison with samples from HMP

loop over the entire sample set:

```
for f in *.fasta.gz
do
     MetaPhlAn2/metaphlan2.py $f --input_type fasta --nproc 4 > ${f%.fasta.gz}_profile.txt
done
```

Merge metaphlan tables:

`MetaPhlAn2/merge_metaphlan_tables.py data/metaphlan_output.txt data/*profile.txt > data/metaphlan_merged.txt`

Alighn:
```
bwa index data/T_forsythia_genome.fasta 
bwa mem -t 8 data/T_forsythia_genome.fasta data/G12_assembly.fna | samtools view -Sb | \
    samtools sort -@ 8 > data/G12_aligned_bwamem.
 ```